<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: María Paz Raveau Morales</em><br>

</div>

# **<center>TAREA 1 MINERÍA DE TEXTO (NLP)</center>**
## <center>Extrayendo contenido útil y visualizando</center>
*20 de julio de 2025*

**Nombre Estudiante**: Cristian Tobar Morales  
**Asignatura**: Procesamiento de Lenguaje Natural 


## **01. INTRODUCCIÓN**

## **02. DESCRIPCIÓN DE LA BASE DE DATOS**

## **03. DESARROLLO**

### **CARGA Y ANÁLISIS EXPLORATORIO DE LOS DATOS**

#### Librerías requeridas

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
import stanza
from nltk.tokenize import sent_tokenize
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

d:\crist\OneDrive\Estudios\Mg Data Science_2024\Trimestre 5\Procesamiento de lenguaje natural\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Cargar e Información inicial del dataset

In [2]:
# Cargar los datos
cabildos = pd.read_excel('data/resultadocabildoprovincial.xlsx')

In [9]:
# Exploración inicial
cabildos.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento


In [7]:
# Información del DataFrame
cabildos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30414 entries, 0 to 30413
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   region_nombre      30414 non-null  object
 1   provincia_nombre   30414 non-null  object
 2   comuna_nombre      30414 non-null  object
 3   cabildo_id         30414 non-null  int64 
 4   cabildo_nombre     30414 non-null  object
 5   cabildo_direccion  30414 non-null  object
 6   num_mesa           30414 non-null  int64 
 7   pregunta_glosa     30414 non-null  object
 8   concepto_nombre    30414 non-null  object
 9   concepto_otro      7121 non-null   object
 10  tipo-eleccion      30414 non-null  object
 11  fundamento         30414 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.8+ MB


### **PRE-PROCESAMIENTO DEL DATAFRAME**

Primeramente, se debe observar de la columna `fundamento`, cuales filas no aportar información relevante para el estudio, dicho caso se procede a eliminar.

In [10]:
cabildos.groupby(['fundamento']).size().reset_index().sort_values(by=0,ascending=False)[0:10]

,fundamento,0
383,.,1655
14999,Sin fundamento,915
19011,sin fundamento,727
67,-,566
14067,SIN FUNDAMENTO,543
15004,Sin fundamento.,410
14936,Sin Fundamento,295
19015,sin fundamentos,214
15005,Sin fundamentos,206
15006,Sin fundamentos.,201


Para poder eliminar todas las filas `sin fundamento`, es crucial convertir todo el texto a minúsculas, debido a que Python es sensible a mayúsculas y minúsculas.

In [12]:
# Transformar a minúscula
cabildos['fundamento'] = cabildos['fundamento'].str.lower()

# Eliminar las puntuaciones: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' 
cabildos['fundamento'] = cabildos['fundamento'].str.translate(str.maketrans('','', string.punctuation))

# Eliminar números
cabildos['fundamento'] = cabildos['fundamento'].str.translate(str.maketrans('', '', string.digits))

# Eliminar espacios en blanco
cabildos['fundamento'] = cabildos['fundamento'].str.strip()

Se pretende tokenizar el texto, la cual consiste en dividir el texto en unidades, comúnmente en palabras.

In [13]:
required = ["punkt", "wordnet", "stopwords", "omw-1.4"]
for pkg in required:
    nltk.download(pkg, quiet=True)
print("Recursos NLTK descargados")

Recursos NLTK descargados


In [18]:
cabildos = cabildos[[len(x.split( )) >3  for x in cabildos.fundamento]]

In [19]:
cabildos.fundamento[0:100].tolist()

['la descentralizacion de ser autonoma por ser zona extrema del pais',
 'toda persona tiene derecho a hacer lo que se le de la gana',
 'chile sin recursos naturales no será capaz de conseguir el desarrollo actualmente chile parece un pais explotado ya no tenemos fuentes primarias no presentamos industrias',
 'ser honestos con nosotros mismos tenemos una cultura e identidad que no nos representa ser uno mismo',
 'se aplica de distintos maneras para priorizar a la clase alta justicia para todos y con igualdad la justicia de nuestra nación no tiene transparencia queremos igualdad y una justicia',
 'tenemos que empezar a avanzar de tal modo de producir un cambio en la democracia incluye libertad integracion justicia etc para poder elegir hay que tener su propia autonomía satisfacer necesidades básicas tener conciencia y tomar decisiones sin pedir ayuda a otras naciones cambiar la con',
 'beneficios para todos recursos y programas debe ser para todos los que necesitan no hay una justicia pa

In [20]:
cabildos.groupby(['fundamento']).size().reset_index().sort_values(by=0,ascending=False)[0:20]

,fundamento,0
272,acta no contiene fundamento,42
5143,el grupo sólo votó y no fundamentó,34
16070,sin descripción en fundamentos,26
5076,el grupo al tratar cada tema consideró estar d...,22
17502,validado por la comisión,20
15417,se entiende que el concepto abordado de forma ...,17
6909,esta votación es la priorización de los encue...,17
21,a favor en contra,17
10889,no se registran datos,17
1853,consagrar y garantizar este derecho fundamenta...,14
